In [1]:
library(shiny)
library(leaflet)
library(lubridate)
library(dplyr)
library(tidyr)

data <- read.csv("Vehicle_Accident_Data.csv")
df <- tidyr::separate(data=data,
                      col=Location,
                      into=c("Latitude", "Longitude"),
                      sep=",",
                      remove=FALSE)
df$Latitude <- stringr::str_replace_all(df$Latitude, "[(]", "")
df$Longitude <- stringr::str_replace_all(df$Longitude, "[)]", "")
df$Latitude <- as.numeric(df$Latitude)
df$Longitude <- as.numeric(df$Longitude)
df$Crashdate = as.Date(df$Crash.Date.Time, format="%m/%d/%Y")

ui <- navbarPage("Vehicle Accident",
                 tabPanel("Interactive Map",
                        tags$h4("Vehicle Accident Location"),
                        leafletOutput("mymap",height = 1000, width = 1000),
                        absolutePanel( fixed = TRUE, draggable = TRUE, top = 100, left = "auto", right = 20, bottom = "auto",
                                      width = 300, height = "auto",
                                      h3("Map Customizing Options"),
                                      dateRangeInput("dateRange", "Date Range", start = min(df$Crashdate), end = max(df$Crashdate), min = min(df$Crashdate), max = max(df$Crashdate)),
                                      checkboxInput("Fatality", "Fatality", value = FALSE),
                                      checkboxInput("Hitandrun", "Hit and Run", value = FALSE)
                                      
                        )  
                ),
                tabPanel("Data Table", 
                         tags$h4("Filtered Data Table"),
                         DT::dataTableOutput("accidentTable")
                        )
    
)
 
server <- function(input,output){

  data <- reactive({
  filter(df, between(Crashdate, as.Date(input$dateRange[1]), as.Date(input$dateRange[2])) & Fatality == input$Fatality & Hit.And.Run == input$Hitandrun)
  })

  output$mymap <- renderLeaflet({
    df <- data()

    m <- leaflet(data = df) %>%
      addTiles(urlTemplate = "//{s}.tiles.mapbox.com/v3/jcheng.map-5ebohr46/{z}/{x}/{y}.png",
               attribution = 'Maps by <a href="http://www.mapbox.com/">Mapbox</a>') %>%
      setView(lng = -97.309341, lat = 32.768799, zoom = 10) %>%
      addMarkers(lng = ~Longitude,
                 lat = ~Latitude)
    m
  })

  output$accidentTable <- DT::renderDataTable({ (data = filter(df, between(Crashdate, as.Date(input$dateRange[1]), as.Date(input$dateRange[2])) & Fatality == input$Fatality & Hit.And.Run == input$Hitandrun))
  DT::datatable(data = data,
                options = list(pageLength = 10),
                rownames = FALSE) 
  })
}

shinyApp(ui = ui, server = server)


ERROR: Error in library(leaflet): there is no package called 'leaflet'
